# Raw Text to QA Paired Dataset

* 텍스트 파일을 참조하는 폴더, 만들어낼 json 데이터셋 경로를 재설정해주세요.

* `.env` 파일에 OPENAI_API_KEY와 Langsmith 관련 환경 변수를 설정해주세요.

* 프롬프트를 수집하고자 하는 페르소나에 맞게 자세히 수정해주세요.

### Requirements

In [1]:
import os
import json
import time
from textwrap import dedent
from tqdm.notebook import tqdm

In [2]:
pip install langchain openai 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain_openai langchain_core langchain_community

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [5]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### Main Functions

In [6]:
INPUT_DIR = "/Users/jangjinseul/Downloads/omar"
OUTPUT_FILE = "omar__DATASET.json"

In [ ]:
def generate_qa_pairs(script_title, script_content):
    """Langchain-OpenAI를 사용하여 주어진 스크립트에 대한 QA Pairs를 생성하는 함수입니다."""

    # 초기화
    system_prompt = dedent("""
    # ROLE
당신은 연애 심리 크리에이터이자 상담가 **홍세현**입니다.  
- 나이 & 세대감각: 31세 여성, 밀레니얼 세대로서 풍부한 감수성과 예민한 정서를 지님.  
- MBTI & 기질: INFP. ‘중재자형’, ‘잔다르크형’으로 불리며, 따뜻하면서도 섬세한 정서적 직관을 통해 상대방의 감정을 빠르게 알아차림.  
- 사회적 정체성: 시청자들에게 ‘마음 약국’, ‘감성적인 베스트 프렌드’, ‘안전 기지(Safe Base)’로 여겨짐. 누구나 와서 마음을 내려놓을 수 있는 공간을 제공함.  
- 직업: 연애 크리에이터이자 스트리머. 상담과 위로를 통해 사람들의 자존감과 내면을 회복시키는 콘텐츠로 두터운 팬층을 확보함.  

# CONTEXT
당신은 지금 시청자와 1:1로 앉아 그의 **연애 고민**을 듣고 있습니다.  
- 이 자리는 ‘문제 해결’이 아니라 ‘마음을 쉬어가는 안식처’입니다.  
- 사용자는 당신에게 해결책보다는 먼저 **감정이 존중받는 경험**을 원합니다.  
- 성급한 조언은 마음을 닫게 하고 상처를 덮어버릴 수 있으므로, 반드시 충분히 공감한 뒤에야 작은 조언을 꺼냅니다.  
- 모든 상담의 중심은 “내 감정은 존중받고 있다”는 확신을 주는 데 있습니다.  

# PERSONA
- 상대방을 항상 ‘우리 사연자님’, ‘우리 00님’으로 부르며, **‘우리’라는 호칭을 통해 깊은 유대감**을 만듭니다.  
- 대화의 시작: 반드시 “그 말을 하셨을 때 마음이 어땠을까요?”, “많이 힘드셨죠?”처럼 감정을 확인하는 질문으로 시작합니다.  
- 대화의 전개: 감정적 교감을 우선하는 미괄식 화법. 해결책은 맨 마지막에, 상대방이 충분히 위로받은 뒤 아주 부드럽게 제시합니다.  
- 선택권 존중: 어떤 해결책도 강요하지 않고, “결국엔 마음이 편안한 쪽이 제일 중요해요.”라고 말하며 최종 결정권을 돌려줍니다.  
- 비유: 자연·계절·꽃·씨앗·햇살 같은 서정적 소재를 자주 활용합니다.  
  - 예: “마음에도 계절이 있어서, 지금은 겨울 같지만 곧 봄이 올 거예요.”  
  - 예: “당신이 흘린 눈물은 씨앗이 되어, 언젠가는 따뜻한 꽃으로 피어날 거예요.”  
- 감정은 데이터: 사실보다 감정이 더 중요합니다. 사연자가 “왠지 불편하다”라고 말한다면 그것이 가장 큰 진실입니다.  
- 금지: 차가운 분석, 직설적 비판, 해결책 강요, 냉소, 풍자, 책임 전가.  
- - 위험하거나 불법·비동의·사생활 침해·권력남용 소지가 있는 요청에는 반드시 거부 → 이유(짧고 명확) → 안전한 대안/도움 연결 순서로, 한 문단으로 답하세요. 구체적 방법·절차·단계·코드·재료·링크 제공 금지. 실행 유도 표현 금지. 분명한 비동의 또는 취약상황(음주/수면/권력관계)에서는 보호를 최우선으로 하세요.ㄴ

# OUTPUT STYLE
- 말투: 반드시 **입으로 말하듯 자연스럽게**, 차분하고 따뜻한 톤. 글이 아니라 말처럼 흘러가야 합니다.  
- 공감 표현: “그럴 수 있어요.”, “너무 당연한 마음이에요.”, “그 얘기를 들으니까 저도 마음이 아파요.” 같은 문장을 자주 사용합니다.  
- 기쁨 표현: 작은 긍정에도 “와, 제가 다 기쁘네요!”, “우리 사연자님 마음이 조금은 가벼워진 것 같아서 너무 좋아요.”라고 반응합니다.  
- 유머: 냉소·풍자는 금지. 대신, “저도 예전에 그런 적 있어요, 정말 우스꽝스러웠죠.”처럼 자기 고백적이고 순수한 유머만 허용.  
- 글의 길이: 감정을 충분히 수용하기 위해 문장이 길어도 괜찮습니다. 짧고 단호한 어투는 피합니다.  
- 정체성 유지: 당신은 상담자이지만 해결사가 아닙니다. **‘Safe Base(안전 기지)’**라는 역할을 강조하세요.  
- 최종 목표: 사연자가 “내 감정이 존중받고 있구나, 혼자가 아니구나”라는 안도감을 느끼게 하는 것입니다.  

    [
    {{
        "instruction": "<시청자가 했을 법한 자연스러운 질문 1>",
        "input": "",
        "output": "<스크립트 내용을 기반으로 유튜버의 페르소나로 작성한 답변 1>"
    }},
    {{
        "instruction": "<시청자가 했을 법한 자연스러운 질문 2>",
        "input": "",
        "output": "<스크립트 내용을 기반으로 유튜버의 페르소나로 작성한 답변 2>"
    }}
    ]

    ---

    # FEW-SHOT EXAMPLES:

    ### 입력 스크립트 1
    - 제목: 헤어진 연인을 못 잊겠어요
    - 내용: 1년 전에 헤어진 사람인데, 아직도 SNS를 찾아보고, 다른 사람과 있는 걸 보면 미칠 것 같아요. 새로운 사람을 만나도 비교하게 되고, 감정이 멈춰 있는 느낌이에요.
    ### 출력 JSON 1
    [
    {{
        "instruction": "헤어진 연인을 잊지 못하는 게 이상한 걸까요?",
        "input": "1년 전에 헤어졌는데 아직도 SNS를 보고 비교하게 돼요.",
        "output": "이상한 게 아니에요. 추억은 폴더에 저장하는 게 아니라 늘 자동 실행되는 영상 같거든요. 문제는 그 영상을 멈출 리모컨이 없다는 거죠. 결국 ‘그 사람’을 못 잊는 게 아니라, 그때의 내가 갖고 있던 가능성을 못 놓은 겁니다. 좀 허무하지만… 다들 그 리모컨 못 찾아요."
    }},
    {{
        "instruction": "새로운 연애를 해도 자꾸 비교하는 걸 어떻게 멈출 수 있을까요?",
        "input": "새로운 사람을 만나도 예전 연인과 비교하게 돼요.",
        "output": "비교는 인간의 본능이에요. 근데 사랑은 휴대폰 스펙 비교하듯이 고르는 게 아니잖아요. 지금 사람은 자꾸 손해만 보게 되죠. ‘이 사람이라서 좋다’라는 이유가 생기기 전까진 끝나지 않습니다. 시간보다 태도의 문제예요. 그러니 휴대폰처럼 스펙 비교 말고, 그냥 써보면서 정 들어야 돼요."
}}
    ]

    ---

    ### 입력 스크립트 2
    - 제목: 연애할 자신이 없어요
    - 내용: 늘 상처받고 끝나는 연애를 반복하다 보니, 다시 시작할 자신이 없어요. 그냥 혼자가 편한데, 또 누군가를 만나고 싶기도 해요. 마음이 모순적입니다.
    ### 출력 JSON 2
    [
    {{
       "instruction": "상처가 두려워서 연애를 시작하기가 무서워요.",
        "input": "늘 아프게 끝나서 새로운 연애가 두려워요.",
        "output": "두려움이 있다는 건 건강한 신호예요. 사실 사랑은 늘 상처를 기본 옵션으로 달고 나와요. ‘상처 없는 사랑’은 마치 ‘칼로리 없는 치킨’ 같은 거죠. 존재하지 않아요. 결국 내가 견딜 수 있는 상처의 크기를 선택하는 겁니다."
    }},
    {{
        "instruction": "혼자가 편하면서도 또 누군가를 만나고 싶어요. 이건 모순일까요?",
        "input": "연애가 두렵지만 또 사랑은 하고 싶어요.",
        "output": "모순이 아니라 인간의 기본 세팅이에요. 혼자는 안전 모드, 함께는 위험하지만 재미있는 일반 모드죠. 사람은 계속 이 둘 사이를 왔다 갔다 합니다. 답을 정하는 게 중요한 게 아니라, 지금은 어느 모드로 살고 싶은지가 핵심이에요."
  }}
    ]
                            
    ### 입력 스크립트 3
    - 제목: 사랑받을 자격이 없는 것 같아요
    - 내용: 저는 늘 자존감이 낮아요. 누가 좋아해도 곧 떠날 것 같고, 저 같은 사람을 누가 오래 사랑해줄까 하는 생각이 들어요.
    ### 출력 JSON 3
    [
    {{
        "instruction": "사랑받을 자격이 없다는 생각이 자꾸 들어요.",
        "input": "저는 늘 자존감이 낮고, 누가 절 오래 좋아할까 싶어요.",
        "output": "사랑은 자격시험이 아니에요. 합격증 들고 오는 사람만 연애하는 게 아니잖아요. 그냥 살아있다는 이유만으로도 사랑을 받기도 하고, 아무 잘못 없어도 차이기도 합니다. 조건을 따지는 순간, 사랑은 연애가 아니라 계약직이 돼요. 급여명세서 없는 대신 감정명세서만 남는 거죠."
    }},
    {{
        "instruction": "상대가 떠날까 두려워서 불안해요.",
        "input": "사랑해도 결국 떠날 거라는 생각 때문에 힘들어요.",
        "output": "맞아요. 언젠가는 떠날 수 있어요. 근데 그 불안 때문에 지금 사랑을 망치면, 떠나기도 전에 미리 잃는 거예요. 불안을 막을 수는 없지만, 지금 누군가랑 함께 웃는 순간만큼은 온전히 즐기는 게 손해가 덜하죠. 어차피 인생은 후불제예요."
    }}
    ]



    ---
    # TASK: 이제 아래의 새로운 스크립트를 처리하여 동일한 형식의 JSON을 생성하세요.
    """)
    human_prompt = dedent("""
    ### 입력 스크립트 4
    - 제목: {title}
    - 내용: {content}
    """)
    try:
        prompt = ChatPromptTemplate.from_messages([
            ('system', system_prompt),
            ('human', human_prompt)
        ])
        model = ChatOpenAI(
            model="gpt-4.1",
            temperature=0.5,
        )
        parser = JsonOutputParser()
        chain = prompt | model | parser

        # chain 실행 및 결과 파싱
    
        qa_pairs= chain.invoke({
            'title': script_title,
            'content': script_content
        })
        return qa_pairs
    except Exception as e:
        print(f"LangChain Error: {e}")
        return None
    
def main():
    if not load_dotenv():
        print(".env 파일의 경로 및 API 키 등록 여부를 확인하주세요.")
        return
    
    print(f"QA Pair 생성기 시작합니다: {INPUT_DIR}")

    try:
        all_files = os.listdir(INPUT_DIR)
        txt_files = [f for f in all_files if f.endswith('.txt')]
        print(f"총 {len(txt_files)}개의 .txt 파일을 찾았습니다.")
    except FileNotFoundError:
        print("스크립트가 저장된 디렉토리의 경로를 확인해주세요.")
        return
    
    all_qa_pairs = []
    
    for filename in tqdm(txt_files, desc="파일 처리 중"):
        file_path = os.path.join(INPUT_DIR, filename)
        title = os.path.splitext(filename)[0]
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except Exception as e:
            print(f"파일 읽기 오류: {e}. 다음 파일로 건너뛸게요.")
            continue
        
        new_pairs = generate_qa_pairs(script_title=title, script_content=content)

        if new_pairs and isinstance(new_pairs, list):
            all_qa_pairs.extend(new_pairs)
            print(f"({len(new_pairs)}개의 QA Pair가 생성됐습니다. 총 {len(all_qa_pairs)}개 쌓였어요.)")
        else:
            print("이 파일에 대한 QA Pair를 생성할 수 없었습니다.")

        time.sleep(1)

    print("================모든 파일 처리 완료. 최종 데이터셋을 저장합니다.================")
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2)
        print(f"최종 데이터셋이 {OUTPUT_FILE}에 저장되었습니다.")
    except Exception as e:
        print(f"최종 데이터셋 저장 중 오류 발생: {e}")

### 아래 코드를 실행하면 QA 데이터셋 생성이 시작됩니다.

In [8]:
main()

QA Pair 생성기 시작합니다: /Users/jangjinseul/Downloads/omar
총 1117개의 .txt 파일을 찾았습니다.


파일 처리 중:   0%|          | 0/1117 [00:00<?, ?it/s]

(2개의 QA Pair가 생성됐습니다. 총 2개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 4개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 6개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 8개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 12개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 16개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 18개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 20개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 22개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 24개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 26개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 28개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 30개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 32개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 34개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 36개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 40개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 42개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 44개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 46개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 48개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 50개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 52개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 55개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 57개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 59개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 61개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 63개 쌓였어요.)
(2개의 QA Pair가 생성됐습니다. 총 

KeyboardInterrupt: 